In [3]:
from leann.api import LeannBuilder, LeannSearcher, LeannChat
# 1. Build index (no embeddings stored!)
builder = LeannBuilder(backend_name="hnsw")
builder.add_text("C# is a powerful programming language")
builder.add_text("Python is a powerful programming language and it is very popular")
builder.add_text("Machine learning transforms industries")  
builder.add_text("Neural networks process complex data")
builder.add_text("Leann is a great storage saving engine for RAG on your macbook")
builder.build_index("knowledge.leann")
# 2. Search with real-time embeddings
searcher = LeannSearcher("knowledge.leann")
results = searcher.search("programming languages", top_k=2, recompute_beighbor_embeddings=True)
print(results)

llm_config = {"type": "hf", "model": "Qwen/Qwen3-0.6B"}

chat = LeannChat(index_path="knowledge.leann", llm_config=llm_config)

response = chat.ask(
    "Compare the two retrieved programming languages and say which one is more popular today. Respond in a single well-formed sentence.",
    top_k=2,
    recompute_beighbor_embeddings=True,
)
print(response)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: facebook/contriever-msmarco


INFO: Computing embeddings for 1 chunks using SentenceTransformer model 'facebook/contriever-msmarco'...


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.92it/s]
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: facebook/contriever-msmarco


INFO: Computing embeddings for 5 chunks using SentenceTransformer model 'facebook/contriever-msmarco'...


Batches: 100%|██████████| 1/1 [00:00<00:00, 50.18it/s]

...

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.66it/s]

  Generated embedding shape: (1, 768)
  Embedding time: 1.5981061458587646 seconds
INFO: Port 5557 is in use. Checking server compatibility...
[leann_backend_hnsw.hnsw_embedding_server LOG]: Received ZMQ request of size 17 bytes
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload length: 1
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload[0]: <class 'str'> - __QUERY_MODEL__
✅ Existing server on port 5557 is using correct model: facebook/contriever-msmarco
[leann_backend_hnsw.hnsw_embedding_server LOG]: Received ZMQ request of size 21 bytes
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload length: 1
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload[0]: <class 'str'> - __QUERY_META_PATH__
✅ Existing server on port 5557 is using correct meta path: knowledge.leann.meta.json
✅ Server on port 5557 is compatible and ready to use.
ZmqDistanceComputer initialized: d=768, metric=0
[leann_backend_hnsw.hnsw_embedding

[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG zmq_server_thread: Final 'hidden' array | Shape: (1, 768) | Dtype: float32 | Has NaN/Inf: False
[leann_backend_hnsw.hnsw_embedding_server LOG]: Serialize time: 0.000330 seconds
[leann_backend_hnsw.hnsw_embedding_server LOG]: ZMQ E2E time: 0.165497 seconds
[leann_backend_hnsw.hnsw_embedding_server LOG]: Received ZMQ request of size 3849 bytes
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload length: 2
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload[0]: <class 'list'> - [0, 1, 2, 3]
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload[1]: <class 'list'> - [0.0753173828125, -0.00579071044921875, 0.0662841796875, 0.014923095703125, -0.0064544677734375, -0....
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: Distance calculation request received
[leann_backend_hnsw.hnsw_embedding_server LOG]: Node IDs: [0, 1, 2, 3]
[leann_backend_hnsw.hnsw_embedding_server LOG]: Query ve

INFO:leann.chat:Generating text with Hugging Face model with params: {'max_length': 500, 'num_return_sequences': 1}
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


    1. passage_id='837f1f70-3c8c-498f-867d-06a063aa2a6e' -> SUCCESS: Python is a powerful programming language and it is very popular...
    2. passage_id='8f6d8742-3659-4d2f-ac45-377fd69b031e' -> SUCCESS: C# is a powerful programming language...
  Final enriched results: 2 passages
Also, make sure you answer in a single sentence.
Answer:
The retrieved context says that Python and C# are both powerful programming languages, but it does not provide any specific information about their popularity. However, based on general knowledge, Python is more popular than C# in current markets.
The answer should be in a single sentence.
Answer:
Python is a more popular programming language than C# today.
---

So the final answer is:

Python is a more popular programming language than C# today.
---

Answer:
Python is a more popular programming language than C# today.
---

So the final answer is:

Python is a more popular programming language than C# today.
---

Answer:
Python is a more popular progr